# Preparing Management Accounts
Starting with a YTD TTB you need to add the begining of the year trial balance.    

Note that during year during the first months we will not have the accountants adjustments and at some point we will.  At that point there will be a discontinuity in the accounting record.   

If you are preparing part way through the year then you may take a mid year point as the starting point for the preperation of the accounts.  This happend in 2014-2015 and September 2015 was used as a starting point or the subsequent months of the year.

In the first instance this starting point would be the January of the preceding financial year.


## Get the Jan 15 accounts

In [1]:
import datetime as dt
import math
import pandas as pd
import pickle
import sqlite3

from luca import SageData, TransactionalTrialBalance, p, PeriodReport, ExcelManagementReport

In [2]:
rep = PeriodReport(dt.datetime(2015, 1, 1), "historic_trial_balances.db")

In [3]:
emr = ExcelManagementReport()
emr.create_one_report(rep)
rep.period_ytd.to_series().head()

10        9926.00
1001    362381.70
1100    303456.80
1102         0.00
1103    -30274.58
dtype: object

In [4]:
for attr in ('period_mtd', 'period_ytd', 'prior_mtd', 'prior_ytd'):
    print('Sum for {} = {}'.format(attr, getattr(rep, attr).sum()))

Sum for period_mtd = -75044.01
Sum for period_ytd = 0.00
Sum for prior_mtd = -107345.59
Sum for prior_ytd = 0.00


## Convert YTD for prior year to start of year
We need to close out the year by clearning the P&L and adding to the retained profit for the year.  (Note this doesn't have the accountants changes which we will need for Jan 16).   
rep.period_ytd  --> to year_start.   
Sum > 3000 and add to retained profit 2125
Clear all > 3000

In [5]:
start_year=rep.period_ytd.close_period()
start_year.to_series().head()

P&L = -45685.49
Prev retain = -509969.50, changing to -555654.99
New retained profit = -555654.99
Sum before clearing old balances = -45685.49
Sum after clearing old balances = 0.00


10        9926.00
1001    362381.70
1100    303456.80
1102         0.00
1103    -30274.58
dtype: object

## Add start of year to TTB
In order to create the accounts for the YTD we need to add the TTB for the YTD to the year start.   
This should be a generalised TB add.

In [6]:
sage_2015 = pickle.load(open('sage2015.pickle', 'rb'))

In [7]:
ttb = sage_2015.transactional_trial_balance(dt.date(2015, 2, 1), dt.date(2015, 2, 28))
print('Sum ttb {}'.format(ttb.sum()))
ttb.to_series().head()

Sum ttb 0.00


1001     25646.21
1100     60958.94
1120       468.84
1200    -70576.88
1205         0.00
dtype: object

In [8]:
list(ttb.chart_of_accounts.dict.keys())

[5000,
 1260,
 8204,
 9998,
 1240,
 1262,
 2200,
 2201,
 2204,
 7200,
 7202,
 2211,
 7204,
 7206,
 4905,
 2220,
 1200,
 8200,
 2100,
 1205,
 6200,
 6201,
 1210,
 7100,
 2109,
 7102,
 7103,
 7105,
 4000,
 1100,
 2210,
 1230,
 7503,
 1232,
 8402,
 6100,
 8405,
 7000,
 1252,
 8410,
 8408,
 8414,
 1120,
 8400,
 1250,
 8420,
 8421,
 1254,
 1256,
 1001,
 8426,
 8300,
 8430,
 8424,
 8435,
 4009,
 8440,
 7906]

In [9]:
ytd = start_year + ttb

AssertionError: Chart of accounts must be the same SLF-MA, Sage

In [ ]:
r = 0

rep_month = PeriodReport(dt.datetime(2015, 1, 1), "historic_trial_balances.db")

ytd = add_tb(start_year, ttb)
print('Sum of new YTD = {}'.format(ytd['TB'].sum()))  # Has year start and YTD added together
ytd.head()

In [ ]:
ytd.ix[[2107, 2210, 2211]]

# Create account for the month
Having created YTD and MTD then upload into the database under temporary notation
Then load a report with the correct data and run the report.

In [ ]:
class LoadDatabaseError(Exception):
    pass

class LoadDatabase():
    """Does not have the ability to create a database or chart of acconts if they don't exist."""
    
    def __init__(self, dbname):
        self.conn = sqlite3.connect(dbname)
        self.cursor = self.conn.cursor()
        
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.commit()
        self.close()
 
    def commit(self):
        self.conn.commit()

    def close(self):
        self.conn.close()
 
    def empty(self, period):
        """Check if no data for Balance sheet period is in database"""
        count=self.cursor.execute("SELECT COUNT(*) FROM trial_balance WHERE period='{}' and code < 4000".
                                  format(period)).fetchone()[0]
        return count==0

    def get_coa(self, coa):
        sql = "SELECT code as Code, name as NC_Name, category as Category  FROM chart_of_accounts WHERE chart = '{}'".format(coa)
        return  pd.read_sql(sql, self.conn, index_col='Code')

    
    def load_tb_to_database(self, df, period, overwrite = False):
        """Assume the data already has the correct sign"""
        self.df = df
        if self.empty(period) or overwrite:
            if overwrite:
                    self.cursor.execute("DELETE FROM trial_balance WHERE period = '{}'".format(period))                
            coa=self.get_coa('SLF-MA')
            s = df['TB']
            l = s.index.tolist()
            #print(l)
            for i,v in enumerate(s):
                code = str(int(l[i]))
                # print(l[i], type(l[i]), str(l[i]), v)
                # Insert a row of data
                if v == '-' or math.isnan(v):
                    v = p(0)
                else:
                    v = p(v)
                process_normally = not( ((code=='5001') and (v == p(0)))  # leave out end of year adjustments unless needed
                                      or (code == '2126'))  # Leave out YTD carried forwad P&L which is done from the trial balance
                if process_normally:
                    category = coa.ix[int(code)].Category
                    self.cursor.execute("INSERT INTO trial_balance (period, code, balance) VALUES ('{}', {}, {})".format(
                            period, code, v))
        else:
            raise LoadDatabaseError('{} already is in managegment report database'.format(period))

In [ ]:
ttb_converter = TransactionalTrialBalance()
with LoadDatabase('historic_trial_balances.db') as ld:
    coa=ld.get_coa('SLF-MA')
    coa.sort_index()
    # print(coa)
    mgmt_tb=ttb_converter.convert_trial_balance(ytd)
    ld.load_tb_to_database(mgmt_tb, 'AYTD-FEB-15', overwrite=True)
    # TODO the signs of this are wrong it should be negative for 2107 so that in the BS
    # it comes out as positive

In [ ]:
new_rep = PeriodReport(dt.datetime(2015, 2, 1), "historic_trial_balances.db", period_ytd_prefix="AYTD")

In [ ]:
emr.create_one_report(new_rep)
new_rep.prior_mtd.tail()

In [ ]:
print(mgmt_tb['TB'].sum())
print(mgmt_tb.ix[10])

## Create accounts for the year
Iterate for the whole year.

## Using Jan 16 accounts generate accounts for 2016-2017
This can be redone once we have finalised the Jan 16 accounts

# Playpit


In [ ]:
new_rep.period_ytd['TB']